In [1]:
import cv2
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands
import numpy as np
import math
from typing import List, Optional, Tuple, Union
import os

In [41]:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue
    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = hands.process(image)
    
    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for num,hand in enumerate(results.multi_hand_landmarks):
        mp_drawing.draw_landmarks(
            image, hand, mp_hands.HAND_CONNECTIONS)
        if get_label(num,hand,results):
            text,coord=get_label(num,hand,results)
            cv2.putText(image,text,coord,cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2,cv2.LINE_AA)
            
            
            
    #hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)  
    mask = cv2.inRange(image, (0, 255, 0), (0, 256,0))
    imask = mask>0
    green = np.zeros_like(image, np.uint8)
    green[imask] = image[imask]
    green[0:125,0:640]=header    
    cv2.imshow('MediaPipe Hands', green)
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()


In [2]:
def get_label(index,hand,results):
    output=None
    for idx,classification in enumerate(results.multi_handedness):
        if classification.classification[0].index==index:
            label=classification.classification[0].label
            score=classification.classification[0].score
            text="{} {}".format(label,round(score,2))
            
            coords=tuple(np.multiply(
                np.array((hand.landmark[mp_hands.HandLandmark.WRIST].x,hand.landmark[mp_hands.HandLandmark.WRIST].y)),
                                    [640,480]).astype(int))
            output=text,coords
    return output
            

In [2]:
overlay=[]
imglist=os.listdir('./resize/')

In [3]:
imglist

['1.jpg', '2.jpg', '3.jpg', '4.jpg', '5.jpg']

In [4]:
for im in imglist:
    image=cv2.imread('./resize/'+im)
    overlay.append(image)


In [5]:
header=overlay[4]


In [13]:
color=(255,0,255)
xp,yp=0,0

In [6]:
# cap.set(3,1280)
# cap.set(4,720)

In [7]:
def get_position(t1,t2,d1,d2):
    if t1<d1 and t2<d2:
        return [1,1]
    elif t1>d1 and t2<d2:
        return [0,1]
    elif t1<d1 and t2>d2:
        return [1,0]
    else:
        return [0,0]

In [24]:
coords=tuple()
imgcanvas=np.zeros((480,640,3),np.uint8)
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    image=cv2.flip(image,1)
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue
    # Flip the image horizontally for a later selfie-view display, and convert
    # the BGR image to RGB.
    image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    results = hands.process(image)
    
    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks:
      for num,hand in enumerate(results.multi_hand_landmarks):
        mp_drawing.draw_landmarks(
            image, hand, mp_hands.HAND_CONNECTIONS)
        
        indexf_coords=tuple(np.multiply(
                np.array((hand.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].x,hand.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP].y)),
                                  [640,480]).astype(int)) 
        middlef_coords=tuple(np.multiply(
                np.array((hand.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].x,hand.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_TIP].y)),
                                  [640,480]).astype(int))
        indexdf_coords=tuple(np.multiply(
                np.array((hand.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].x,hand.landmark[mp_hands.HandLandmark.INDEX_FINGER_DIP].y)),
                                  [640,480]).astype(int)) 
        middledf_coords=tuple(np.multiply(
                np.array((hand.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].x,hand.landmark[mp_hands.HandLandmark.MIDDLE_FINGER_DIP].y)),
                                  [640,480]).astype(int))
        finger=get_position(round(indexf_coords[1]),round(middlef_coords[1]),round(indexdf_coords[1]),round(middledf_coords[1]))
        
        if finger[0] and finger[1]==False:
            cv2.circle(image,(indexf_coords[0],indexf_coords[1]),15,color,cv2.FILLED)
            if xp==0 and yp==0:
                xp,yp=indexf_coords[0],indexf_coords[1]
                
            
            xp,yp=indexf_coords[0],indexf_coords[1]
            if color==(0,0,0):
                cv2.line(image,(xp,yp),(indexf_coords[0],indexf_coords[1]),color,50)
                cv2.line(imgcanvas,(xp,yp),(indexf_coords[0],indexf_coords[1]),color,50)

            else: 
                cv2.line(image,(xp,yp),(indexf_coords[0],indexf_coords[1]),color,15)
                cv2.line(imgcanvas,(xp,yp),(indexf_coords[0],indexf_coords[1]),color,15)

                
            
        if finger[0] and finger[1]:
            xp,yp=0,0
            if indexf_coords[1]<125:
#                 print(indexf_coords[0])
                if 0<indexf_coords[0]<130:
                    header=overlay[0]
                    color=(255,0,0)
                elif 130<indexf_coords[0]<274:
                    header=overlay[1]
                    color=(0,0,255)
                elif 274<indexf_coords[0]<360:
                    header=overlay[2]
                    color=(255,0,255)
                elif 360<indexf_coords[0]<512:
                    header=overlay[3]
                    color=(0,255,0)
                elif 512<indexf_coords[0]<640:
                    header=overlay[4]
                    color=(0,0,0)
            cv2.rectangle(image,(indexf_coords[0],indexf_coords[1]-15),(middlef_coords[0],middlef_coords[1]+15),color,cv2.FILLED)
                    
                
    imggray=cv2.cvtColor(imgcanvas,cv2.COLOR_BGR2GRAY)
    _,imgInv=cv2.threshold(imggray,50,255,cv2.THRESH_BINARY_INV)
    imgInv=cv2.cvtColor(imgInv,cv2.COLOR_GRAY2BGR)
    image=cv2.bitwise_and(image,imgInv)
    image=cv2.bitwise_or(image,imgcanvas)
    image[0:125,0:640]=header  
    #image=cv2.addWeighted(image,0.5,imgcanvas,0.5,0)
    cv2.imshow('MediaPipe Hands', image)
    key = cv2.waitKey(1)
    if key == 27:
        break

cap.release()
cv2.destroyAllWindows()
